<a href="https://colab.research.google.com/github/gabriel-reis12/Web-Scraping/blob/main/FII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalação das Lib
!pip install requests --quiet
!pip install pandas --quiet
!pip install beautifulsoup4 --quiet

In [ ]:
#Importando as Bibliotecas
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [ ]:
#url do site desejado
url = 'https://www.fundamentus.com.br/fii_resultado.php'

In [ ]:
#Obtendo conteúdo da Página Fundamentus
#Verificar o User-Agent - Digitar no google: "My user Agent"

headers = {
            "User-Agent"      : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
            "Accept"          : "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
            "Accept-Language" : "en-US,en;q=0.5",
            "DNT"             : "1",
            "Connection"      : "close"
}

deira = data = requests.get(url, headers=headers, timeout=5).text
soup = BeautifulSoup(data, 'html.parser')

In [ ]:
#Procurando a Tabela de FII
table = soup.find('table')

In [ ]:
#Criar Data Frame Pandas
df = pd.DataFrame(columns = ['Papel', 'Segmento', 'Cotação', 'FFO Yield', 'Dividend_Yield', 'P_VP','Valor_de_Mercado', 'Liquidez', 'Qtd Imóveis', 'Preço do m2', 'Aluguel do m2', 'Cap Rate', 'Vacância Média'])

In [ ]:
#Puxando a tabela do Site
for row in table.tbody.find_all('tr'):
  columns = row.find_all('td')
  if(columns != []):
    papel = columns[0].text.strip(" ")
    segmento = columns[1].text.strip(" ")
    cotacao = columns[2].text.strip(" ")
    ffo_yield = columns[3].text.strip(" ")
    dividend_yield = columns[4].text.strip(" ")
    pvp = columns[5].text.strip(" ")
    valor = columns[6].text.strip(" ")
    liquidez = columns[7].text.strip(" ")
    qtd_imoveis = columns[8].text.strip(" ")
    preco = columns[9].text.strip(" ")
    aluguel = columns[10].text.strip(" ")
    cap_rate = columns[11].text.strip(" ")
    vacancia = columns[12].text.strip(" ")
    df = pd.concat([df, pd.DataFrame.from_records([{"Papel":papel, "Segmento":segmento, "Cotação":cotacao, "FFO Yield": ffo_yield, "Dividend_Yield":dividend_yield,"P_VP":pvp,
                                                    "Valor_de_Mercado":valor,"Liquidez": liquidez,"Qtd Imóveis":qtd_imoveis, "Preço do m2":preco, "Aluguel do m2":aluguel,
                                                    "Cap Rate": cap_rate, "Vacância Média": vacancia}])])

In [ ]:
#Criando nosso banco de dados somente para as colunas que iremos fazer os filtros
bd = df[['Papel', 'Segmento', 'Dividend_Yield', 'P_VP', 'Valor_de_Mercado', 'Liquidez']]

In [ ]:
#Arrumando as variáves de texto para "Int" e "Float"

#P/VP
bd['P_VP'] = [x.replace(',','.') for x in bd['P_VP']]
bd = bd.astype({"P_VP":float})

#Valor de Mercado
bd['Valor_de_Mercado'] = [x.replace('.','') for x in bd['Valor_de_Mercado']]
bd = bd.astype({"Valor_de_Mercado":int}, errors='ignore')

#Liquidez
bd['Liquidez'] = [x.replace('.','') for x in bd['Liquidez']]
bd = bd.astype({"Liquidez":int}, errors='ignore')

<ipython-input-295-1adafb366bab>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bd['P_VP'] = [x.replace(',','.') for x in bd['P_VP']]


In [ ]:
#Definindo parâmetros P/VP
bd.query('P_VP>0.85', inplace=True)
bd.query('P_VP<1.05', inplace=True)

In [ ]:
#Definindo parâmetros Liquidez
bd.query('Liquidez>400000', inplace=True)

In [ ]:
#Definindo parâmetros Valor de Mercado
bd.query('Valor_de_Mercado>500000000', inplace=True)

In [ ]:
#Transformando Dividend Yield em Float
def converter_para_float(x):
    x = x.replace(',', '.')
    x = x.replace('%', '')
    return float(x)

bd['Dividend_Yield'] = bd['Dividend_Yield'].apply(converter_para_float)

In [ ]:
#Definindo parâmetros Dividend Yield
bd.query('Dividend_Yield >8', inplace=True)
bd.query('Dividend_Yield <12', inplace=True)

In [ ]:
bd

,Papel,Segmento,Dividend_Yield,P_VP,Valor_de_Mercado,Liquidez
0,BCFF11,Títulos e Val. Mob.,9.45,0.93,1837600000,3359670
0,BTLG11,Híbrido,8.28,1.04,2936740000,7916590
0,CPTS11,Títulos e Val. Mob.,10.70,0.94,2711070000,6695720
0,GALG11,Híbrido,9.41,1.00,523116000,2793600
0,GGRC11,Logística,10.37,1.01,1029470000,1413910
...,...,...,...,...,...,...
0,RZTR11,Híbrido,10.78,1.03,1133420000,2848120
0,VCJR11,Títulos e Val. Mob.,11.01,0.97,1380670000,1883900
0,VGIP11,Títulos e Val. Mob.,11.23,0.96,1049060000,1873850
0,VRTA11,Títulos e Val. Mob.,10.80,0.93,1354980000,2176760
